In [25]:
import pandas as pd
import re
from bs4 import BeautifulSoup

In [21]:
data = pd.read_csv('../datafiles/ground_truth.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2062 entries, 0 to 2061
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   App id                   2062 non-null   object
 1   Review id                2062 non-null   object
 2   Sentence id              2062 non-null   int64 
 3   Sentence content         2062 non-null   object
 4   Feature (Positive)       291 non-null    object
 5   Feature (Neutral)        638 non-null    object
 6   Feature (Negative)       110 non-null    object
 7   Feature (All Annotated)  971 non-null    object
dtypes: int64(1), object(7)
memory usage: 129.0+ KB


In [29]:
def pre_process(text):
  text = BeautifulSoup(str(text)).get_text()
  text = re.sub("[^a-zA-Z]", " ", text)
  text = text.lower()
  tokens = text.split()
  return tokens

In [30]:
def generate_bio_tags(sentence, keywords):
    clean_content_tokens = pre_process(sentence)
    
    tags = ['O'] * len(clean_content_tokens)
    
    for keyword in keywords:
        keyword_tokens = pre_process(keyword)
        for i in range(len(clean_content_tokens)):
            if clean_content_tokens[i:i+len(keyword_tokens)] == keyword_tokens:
                tags[i] = 'B'
                for j in range(1, len(keyword_tokens)):
                    tags[i+j] = 'I'
                break
    
    return clean_content_tokens, tags

In [32]:
tags_column=[]
clean_content=[]
for i, row in data.iterrows():
    keywords=[]
    # print(row['Feature (All Annotated)'])
    if row['Feature (All Annotated)'] is not None:
        keywords = str(row['Feature (All Annotated)']).split(';')
    clean_content_tokens, tag_sequence = generate_bio_tags(row['Sentence content'], keywords)
    tags_column.append(tag_sequence)
    clean_content.append(clean_content_tokens)

print(len(tags_column))
print(len(clean_content))

C:\Users\aakash.sorathiya\AppData\Local\Temp\ipykernel_29632\152839484.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(str(text)).get_text()


2062
2062


In [33]:
new_df = pd.DataFrame({'clean_content': clean_content, 'tags': tags_column})
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2062 entries, 0 to 2061
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   clean_content  2062 non-null   object
 1   tags           2062 non-null   object
dtypes: object(2)
memory usage: 32.3+ KB


In [34]:
final_df = pd.concat([data, new_df], axis=1)
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2062 entries, 0 to 2061
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   App id                   2062 non-null   object
 1   Review id                2062 non-null   object
 2   Sentence id              2062 non-null   int64 
 3   Sentence content         2062 non-null   object
 4   Feature (Positive)       291 non-null    object
 5   Feature (Neutral)        638 non-null    object
 6   Feature (Negative)       110 non-null    object
 7   Feature (All Annotated)  971 non-null    object
 8   clean_content            2062 non-null   object
 9   tags                     2062 non-null   object
dtypes: int64(1), object(9)
memory usage: 161.2+ KB


In [35]:
final_df.to_csv('../datafiles/true_tags.csv')